##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Python 函数组件教程


注：我们建议在 Colab 笔记本中运行本教程，无需进行设置！只需点击“在 Google Colab 中运行”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/tfx/python_function_component"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/python_function_component.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/download_logo_32px.png"> 下载笔记本</a></td>
</table></div>

此笔记本包含有关如何在 TFX InteractiveContext 和本地编排的 TFX 流水线中创建和运行 Python 函数组件的示例。

有关更多上下文和信息，请参阅 TFX 文档站点上的[自定义 Python 函数组件](https://tensorflow.google.cn/tfx/guide/custom_function_component)页面。

## 设置

我们将首先安装 TFX 并导入必要的模块。TFX 需要 Python 3。

### 检查系统 Python 版本


In [ ]:
import sys
sys.version

### 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### 安装 TFX

**注：在 Google Colab 中，由于软件包更新，第一次运行此代码单元时必须重新启动运行时 (Runtime &gt; Restart runtime ...)。**

In [ ]:
!pip install tfx

### 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。

In [ ]:
!pip uninstall shapely -y

## 是否已重新启动运行时？

如果您使用的是 Google Colab，首次运行上面的代码单元时，必须重新启动运行时 (Runtime &gt; Restart runtime ...)。这样做的原因是 Colab 加载软件包的方式。

### 导入软件包

我们导入 TFX 并检查其版本。

In [ ]:
# Check version
from tfx import v1 as tfx
tfx.__version__

## 自定义 Python 函数组件

在本部分中，我们将从 Python 函数创建组件。我们不会处理任何真正的 ML 问题 - 这些简单的函数只是用来说明 Python 函数组件的开发过程。

有关更多文档，请参阅[基于 Python 函数的组件指南](https://tensorflow.google.cn/tfx/guide/custom_function_component)。

### 创建 Python 自定义组件

我们首先编写一个生成一些虚拟数据的函数。这将写入其自己的 Python 模块文件。

In [ ]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

接下来，我们编写第二个组件，该组件使用生成的虚拟数据。我们将只计算数据的哈希值并将其返回。

In [ ]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

### 使用 InteractiveContext 在笔记本中运行

现在，我们将演示如何在 TFX InteractiveContext 中使用我们的新组件。

有关使用 TFX 笔记本 InteractiveContext 能够做什么的更多信息，请参阅笔记本内的 [TFX Keras 组件教程](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras)。

In [ ]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### 构造 InteractiveContext

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### 使用 `context.run()` 以交互方式运行组件

接下来，我们使用 `context.run()` 在笔记本内以交互方式运行我们的组件。我们的 consumer 组件使用 generator 组件的输出。

In [ ]:
generator = MyGenerator()
context.run(generator)

In [ ]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

执行后，我们可以检查磁盘上 consumer 组件的“hash”输出工件的内容。

In [ ]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

就是这样，您现在已经编写并执行了自己的自定义组件！

### 编写流水线定义

接下来，我们将使用这些相同的组件创建一个流水线。虽然在笔记本内使用 `InteractiveContext` 可以很好地进行实验，但定义流水线可以让您将流水线部署在本地或远程运行程序上，以供生产使用。

在此处，我们将演示您的机器上本地运行的 LocalDagRunner 的用法。对于生产执行，Airflow 或 Kubeflow 运行程序可能更合适。

#### 构造流水线

In [ ]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### 使用 `LocalDagRunner` 运行流水线

In [ ]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

我们可以检查执行此流水线所生成的输出工件。

In [ ]:
!find {PIPELINE_ROOT}

您现在已经编写了自己的自定义组件并在 LocalDagRunner 上编排了它们的执行！有关后续步骤，请查看 [TFX 网站](https://tensorflow.google.cn/tfx)上的其他教程和指南。